In [1]:
%reload_ext autoreload
%autoreload 2

In [13]:
from more_itertools import unique_justseen

In [18]:
from test_queries import _get_config, test_get_engine
from nbc_analysis.utils.debug_utils import runit
from pathlib import Path
import pandas as pd
from pandas.util import hash_array
import numpy as np
from toolz import groupby, pluck
import json
from nbc_analysis.utils.dim_utils import mk_hash

2019-11-08 14:35:21,806|INFO|nbc_analysis.utils.db_utils|start get db, cfg={'db_f': PosixPath('/Users/wmcabee/DATA/NBC/test_db.sqlite'), 'connect_str': 'sqlite:////%s'}
2019-11-08 14:35:21,807|INFO|nbc_analysis.utils.db_utils|end get_db,connect_str=sqlite://///Users/wmcabee/DATA/NBC/test_db.sqlite


In [77]:
SQL = '''

    select 
		mpid,
		platform, 
		data_connection_type,
		nbc_profile,
		city,
        network,
        show,
        genre,
		f.event_start_local_dt,
        f.event_start_dt,
        f._last_upd_dt
    from F_VIDEO_END f
    join DIM_VIDEO vid on f.video_key = vid.video_key
    join DIM_HOUR_IN_WEEK how on f.hour_in_week_key = how.hour_in_week_key
	join DIM_DAY day_utc on f.day_utc_key = day_utc.day_key
	join DIM_DAY_LOC day_loc on f.day_key_loc = day_loc.day_key_loc
	join DIM_EVENT_TYPE evt on f.event_type_key = evt.event_type_key
	join DIM_NETWORK net on f.network_key = net.network_key
	join DIM_PLATFORM plat on  f.platform_key = plat.platform_key
	join DIM_PROFILE prof on f.profile_key = prof.profile_key
	join DIM_END_TYPE endt on f.end_type_key = endt.end_type_key
    where 1=1
    and f.mpid = :mpid
	order by f._last_upd_dt desc
    '''
def get_profile_diffs(mpid, cols):
    df = pd.read_sql(sql=SQL, con=ENGINE, params={'mpid': mpid})
    df['_hash'] = mk_hash(df, cols=cols)
    df['changed'] = (df._hash != df._hash.shift(-1))
    df = df[df.changed].copy()
    df = df.drop(columns=['_hash', '_last_upd_dt', 'changed'])

    return df

In [78]:
ENGINE = test_get_engine()

2019-11-08 14:38:49,464|INFO|nbc_analysis.utils.db_utils|start get db, cfg={'db_f': PosixPath('/Users/wmcabee/DATA/NBC/test_db.sqlite'), 'connect_str': 'sqlite:////%s'}
2019-11-08 14:38:49,465|INFO|nbc_analysis.utils.db_utils|end get_db,connect_str=sqlite://///Users/wmcabee/DATA/NBC/test_db.sqlite


## Example 1:  

In [82]:
with pd.option_context('max_rows', None):
    df = get_profile_diffs(mpid=2882228801219022660, cols=['platform','data_connection_type','nbc_profile','city', 'network'])
    display(df)

,mpid,platform,data_connection_type,nbc_profile,city,network,show,genre,event_start_local_dt,event_start_dt
0,2882228801219022660,iOS,offline,Free,Nashville,2600:1004:b140::/43,Days of our Lives,Drama,2019-08-18T18:06:10.951999-05:00,20190818T23:06:10.951999Z
19,2882228801219022660,iOS,mobile,Free,Nashville,2600:1004:b140::/43,Days of our Lives,Drama,2019-08-18T15:13:16.529000-05:00,20190818T20:13:16.529000Z
20,2882228801219022660,iOS,offline,Free,Nashville,2600:1004:b140::/43,Days of our Lives,Drama,2019-08-18T15:11:33.240999-05:00,20190818T20:11:33.240999Z
24,2882228801219022660,iOS,mobile,Free,Murfreesboro,2600:1004:b120::/44,Days of our Lives,Drama,2019-08-18T10:44:08.000999-05:00,20190818T15:44:08.000999Z
25,2882228801219022660,iOS,mobile,Free,Knoxville,174.195.8.0/21,Days of our Lives,Drama,2019-08-18T10:44:47.203000-04:00,20190818T14:44:47.203000Z
26,2882228801219022660,iOS,mobile,Free,Murfreesboro,2600:1004:b120::/44,Days of our Lives,Drama,2019-08-18T09:29:09.687999-05:00,20190818T14:29:09.687999Z
45,2882228801219022660,iOS,mobile,Free,Knoxville,2600:1004:b100::/43,Days of our Lives,Drama,2019-08-17T14:37:10.469000-04:00,20190817T18:37:10.469000Z
46,2882228801219022660,iOS,offline,Free,Knoxville,2600:1004:b100::/43,Days of our Lives,Drama,2019-08-17T14:25:34.053999-04:00,20190817T18:25:34.053999Z
47,2882228801219022660,iOS,mobile,Free,Nashville,2600:1004:b140::/43,Days of our Lives,Drama,2019-08-17T13:10:43.265000-05:00,20190817T18:10:43.265000Z
48,2882228801219022660,iOS,offline,Free,Nashville,2600:1004:b140::/43,Days of our Lives,Drama,2019-08-17T13:08:31.974999-05:00,20190817T18:08:31.974999Z


## Example 2:

In [83]:
with pd.option_context('max_rows', None):
    df = get_profile_diffs(mpid=2882228801219022660, cols=['data_connection_type'])
    display(df)

,mpid,platform,data_connection_type,nbc_profile,city,network,show,genre,event_start_local_dt,event_start_dt
0,2882228801219022660,iOS,offline,Free,Nashville,2600:1004:b140::/43,Days of our Lives,Drama,2019-08-18T18:06:10.951999-05:00,20190818T23:06:10.951999Z
19,2882228801219022660,iOS,mobile,Free,Nashville,2600:1004:b140::/43,Days of our Lives,Drama,2019-08-18T15:13:16.529000-05:00,20190818T20:13:16.529000Z
20,2882228801219022660,iOS,offline,Free,Nashville,2600:1004:b140::/43,Days of our Lives,Drama,2019-08-18T15:11:33.240999-05:00,20190818T20:11:33.240999Z
45,2882228801219022660,iOS,mobile,Free,Knoxville,2600:1004:b100::/43,Days of our Lives,Drama,2019-08-17T14:37:10.469000-04:00,20190817T18:37:10.469000Z
46,2882228801219022660,iOS,offline,Free,Knoxville,2600:1004:b100::/43,Days of our Lives,Drama,2019-08-17T14:25:34.053999-04:00,20190817T18:25:34.053999Z
47,2882228801219022660,iOS,mobile,Free,Nashville,2600:1004:b140::/43,Days of our Lives,Drama,2019-08-17T13:10:43.265000-05:00,20190817T18:10:43.265000Z
48,2882228801219022660,iOS,offline,Free,Nashville,2600:1004:b140::/43,Days of our Lives,Drama,2019-08-17T13:08:31.974999-05:00,20190817T18:08:31.974999Z
92,2882228801219022660,iOS,mobile,Free,Nashville,2600:1004:b140::/43,Days of our Lives,Drama,2019-08-16T16:28:16.688999-05:00,20190816T21:28:16.688999Z
93,2882228801219022660,iOS,offline,Free,Nashville,2600:1004:b140::/43,Days of our Lives,Drama,2019-08-16T15:49:57.460999-05:00,20190816T20:49:57.460999Z
103,2882228801219022660,iOS,mobile,Free,McMinnville,2600:1004:b160::/44,Days of our Lives,Drama,2019-08-16T14:38:33.494999-05:00,20190816T19:38:33.494999Z


## Example 3: 

In [88]:
with pd.option_context('max_rows', None):
    df = get_profile_diffs(mpid=594429747067128960, cols=['event_start_dt'])
    display(df)

,mpid,platform,data_connection_type,nbc_profile,city,network,show,genre,event_start_local_dt,event_start_dt
0,594429747067128960,Android,wifi,Unauthenticated,Baltimore,2601:147:300::/41,Access,News and Information,2019-08-17T06:17:32.819999-04:00,20190817T10:17:32.819999Z
1,594429747067128960,Android,wifi,Unauthenticated,Baltimore,2601:147:300::/41,Tia & Tamera,Reality and Game Show,2019-08-17T06:15:21.448999-04:00,20190817T10:15:21.448999Z
2,594429747067128960,Android,wifi,Unauthenticated,Baltimore,2601:147:300::/41,Tia & Tamera,Reality and Game Show,2019-08-17T06:14:37.394000-04:00,20190817T10:14:37.394000Z
3,594429747067128960,Android,wifi,Unauthenticated,Baltimore,2601:147:300::/41,Iron Man,Drama,2019-08-17T06:06:58.578999-04:00,20190817T10:06:58.578999Z
4,594429747067128960,Android,wifi,Unauthenticated,Baltimore,2601:147:300::/41,Iron Man,Drama,2019-08-16T23:37:07.311000-04:00,20190817T03:37:07.311000Z
5,594429747067128960,Android,wifi,Unauthenticated,Baltimore,2601:147:300::/41,Iron Man,Drama,2019-08-16T23:01:01.181999-04:00,20190817T03:01:01.181999Z
6,594429747067128960,Android,wifi,Unauthenticated,Baltimore,2601:147:300::/41,Iron Man,Drama,2019-08-16T22:44:07.042999-04:00,20190817T02:44:07.042999Z
7,594429747067128960,Android,wifi,Unauthenticated,Baltimore,2601:147:300::/41,This Is Us,Drama,2019-08-16T22:42:34.874000-04:00,20190817T02:42:34.874000Z
8,594429747067128960,Android,wifi,Unauthenticated,Baltimore,2601:147:300::/41,This Is Us,Drama,2019-08-16T22:10:55.434000-04:00,20190817T02:10:55.434000Z
9,594429747067128960,Android,wifi,Unauthenticated,Baltimore,2601:147:300::/41,This Is Us,Drama,2019-08-16T21:48:10.448999-04:00,20190817T01:48:10.448999Z
